## Using the LQT-Augmented Controller

### All the necessary imports

### Creating the LQT Controller

In [ ]:
# Initialize controller
controller = LQTController(
    state_dim=6,  # 3 variables (pos, vel, acc) × 2 dimensions
    control_dim=2,  # 2D control input
    dt=1.0/(T * l),  # Your DMP timestep
    T=num_timesteps  # Total trajectory timesteps
)

### Creating the DMP integrator

In [ ]:
# Initialize DMP with controller
dmp_integrator = DMPIntegrator(rbf='gaussian')
dmp_integrator.set_controller(controller)

# Initialize DMP parameters
dmp_params = DMPParameters(
    N=5,  # Number of basis functions
    tau=1.0,  # Time scaling
    dt=0.01,  # Timestep
    Dof=2,   # Degrees of freedom (2D)
    scale=None,
    a_z=25  # Default alpha_z
)

### Use in Neuroscribe

In [ ]:
# Example usage
# Assuming your input is EEG data of shape [batch_size, channels, time]
eeg_input = torch.randn(32, 5, 384)  # Example batch of EEG data
y0 = torch.zeros(32, 2)  # Initial positions
goal = torch.ones(32, 2)  # Target positions

# Get TSFEEGNet features
tsf_output = model.TSFEEGNet(eeg_input)
s = model.L(tsf_output)

# Generate trajectory with LQT control
X, dX, ddX = dmp_integrator.forward(
    inputs=tsf_output,
    parameters=dmp_params.data_tensor,
    param_gradients=dmp_params.grad_tensor,
    scaling=None,
    y0=y0,
    dy0=None,  # Will default to zeros + 0.01
    goal=goal,
    s=s
)